## Analyzing the weather impact on the sales of a San Francisco based bike rental company.

#### Motivation: Bike sharing programs are proven to be a boon with cutting down the traffic congestion and in turn lowering the pollution levels. The bike sharing programs today have a recreational, environmental impacts and have a positive influence on an individual’s health.  
 
#### Problem: We are analyzing the data from a popular bike sharing company to see if the daily weather conditions have any impact on the number of trips recorded. We are going to come up with few recommendations which might help the business prevent declining sales.  
 

#### DataSet : We have used the data for a San Francisco based bike rental company https://www.bikerentalsanfrancisco.com for our analysis. The data was collected from Kaggle where we had separate datasets for trips and weather based on city and date. We performed data exploration and feature engineering based on what was important for this experiment.  
 
The list of variables that we have retained in our final dataset are:  
 
1. Date 2. Mean Weather Conditions (mean temperature, mean humidity, mean visibility, mean wind speed, precipitation) 3. Weather Events (for eg. Rain, Thunderstorm) 4. Trips (no of trips in a day), 5. Date features (month, year, weekday number). 
 
Link to Kaggle dataset: https://www.kaggle.com/benhamner/sf-bay-area-bike-share 

In [23]:
#importing the libraries
import os
import pandas as pd
import numpy as np
import glob
import re
from datetime import datetime
import scipy

In [11]:
pd.options.display.max_columns = 999

### dataset 1: Trip

In [4]:
trip = pd.read_csv(r"C:\Users\Ujjwal Singh\Desktop\Pandas\sf-bay-area-bike-share\trip.csv")

In [5]:
trip.rename(columns = {'id':'trip_id'}, inplace=True)

In [6]:
len(trip)

669959

In [7]:
trip.head()

,trip_id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103


In [8]:
#converting start_date column from string to date dtype.
trip.start_date = pd.to_datetime(trip.start_date, format='%m/%d/%Y %H:%M')
#Parsing date
trip['date'] = trip.start_date.dt.date
#converting string 'date' back to date format
trip.date = pd.to_datetime(trip.date)

In [9]:
### feature engineering to calculate the number of trips on each date
trip_cnt = trip.groupby(['date'], as_index= False).agg({'trip_id':'count'})
trip_cnt.rename(columns = {'trip_id':'trips'}, inplace=True)
trip_cnt.head()

,date,trips
0,2013-08-29,748
1,2013-08-30,714
2,2013-08-31,640
3,2013-09-01,706
4,2013-09-02,661


In [10]:
### no of trips based on date and zip code
trip_cnt_zip = trip.groupby(['date','zip_code'], as_index= False).agg({'trip_id':'count'})
trip_cnt_zip.rename(columns = {'trip_id':'trips'}, inplace=True)
#filtering with only the zip codes (san fransisco) avaliable in weather dataset.
trip_cnt_zip2 = trip_cnt_zip.loc[trip_cnt_zip.zip_code.isin(['94107', '94063', '94301', '94041', '95113'])]
trip_cnt_zip2.head()

,date,zip_code,trips
41,2013-08-29,94041,5
44,2013-08-29,94063,2
51,2013-08-29,94107,37
72,2013-08-29,94301,4
108,2013-08-29,95113,2


### dataset 2: Weather

In [12]:
weather = pd.read_csv(r"C:\Users\Ujjwal Singh\Desktop\Pandas\sf-bay-area-bike-share\weather.csv")

In [13]:
weather.zip_code.unique()

array([94107, 94063, 94301, 94041, 95113], dtype=int64)

In [14]:
weather.describe()

,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure_inches,mean_sea_level_pressure_inches,min_sea_level_pressure_inches,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,cloud_cover,wind_dir_degrees,zip_code
count,3661.000000,3661.000000,3661.000000,3611.000000,3611.000000,3611.000000,3611.000000,3611.000000,3611.000000,3664.000000,3664.000000,3664.000000,3652.000000,3652.000000,3652.000000,3664.000000,3664.000000,2766.000000,3664.000000,3664.000000,3665.000000
mean,70.580989,61.348812,51.947282,53.149820,48.794794,44.306840,85.446968,67.256993,46.458045,30.074997,30.022290,29.965835,10.555312,9.822015,8.229737,16.398472,5.962063,22.299349,2.784389,266.605895,94325.000000
std,8.385572,7.234397,7.441444,6.885449,7.822281,9.355432,9.226972,10.945591,14.434784,0.134492,0.130795,0.133476,2.193828,1.469894,2.862132,7.787588,3.340603,9.618813,2.276401,102.047645,404.615212
min,44.000000,38.000000,25.000000,20.000000,13.000000,2.000000,24.000000,24.000000,4.000000,29.500000,29.430000,28.980000,5.000000,4.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,94041.000000
25%,64.000000,56.000000,47.000000,49.000000,44.000000,39.000000,82.000000,62.000000,37.000000,29.980000,29.930000,29.880000,10.000000,10.000000,7.000000,13.000000,3.000000,17.000000,1.000000,254.000000,94063.000000
50%,70.000000,61.000000,53.000000,54.000000,49.000000,46.000000,86.000000,68.000000,48.000000,30.060000,30.010000,29.950000,10.000000,10.000000,10.000000,16.000000,6.000000,22.000000,3.000000,297.000000,94107.000000
75%,77.000000,67.000000,58.000000,58.000000,55.000000,52.000000,93.000000,74.000000,56.000000,30.170000,30.110000,30.050000,10.000000,10.000000,10.000000,20.000000,8.000000,25.000000,5.000000,330.000000,94301.000000
max,102.000000,84.000000,75.000000,68.000000,65.000000,63.000000,100.000000,96.000000,93.000000,30.650000,30.410000,30.370000,20.000000,20.000000,20.000000,128.000000,23.000000,114.000000,8.000000,2772.000000,95113.000000


In [15]:
#dropping all the extreme columns which are not needed in the model

weather = weather.drop(['max_dew_point_f','mean_dew_point_f','min_dew_point_f','max_sea_level_pressure_inches','mean_sea_level_pressure_inches',
                        'min_sea_level_pressure_inches','max_wind_Speed_mph','max_gust_speed_mph','wind_dir_degrees',
                        'max_temperature_f','min_temperature_f','max_humidity','min_humidity','max_visibility_miles','min_visibility_miles',
                        'cloud_cover'], axis =1)

In [16]:
weather.date = pd.to_datetime(weather.date, format='%m/%d/%Y')

In [17]:
weather.head()

,date,mean_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,events,zip_code
0,2013-08-29,68.0,75.0,10.0,11.0,0,NaN,94107
1,2013-08-30,69.0,70.0,10.0,13.0,0,NaN,94107
2,2013-08-31,64.0,75.0,10.0,15.0,0,NaN,94107
3,2013-09-01,66.0,68.0,10.0,13.0,0,NaN,94107
4,2013-09-02,69.0,77.0,10.0,12.0,0,NaN,94107


In [18]:
#events shows us the weather condition of that day and zip code
weather.events.unique()

array([nan, 'Fog', 'Rain', 'Fog-Rain', 'rain', 'Rain-Thunderstorm'],
      dtype=object)

In [19]:
#imputing null values from events as Normal
weather.loc[weather.events == 'rain', 'events'] = "Rain"
weather.loc[weather.events.isnull(), 'events'] = "Normal"

In [20]:
weather.events.unique()

array(['Normal', 'Fog', 'Rain', 'Fog-Rain', 'Rain-Thunderstorm'],
      dtype=object)

In [21]:
weather.isnull().sum()

date                      0
mean_temperature_f        4
mean_humidity            54
mean_visibility_miles    13
mean_wind_speed_mph       1
precipitation_inches      1
events                    0
zip_code                  0
dtype: int64

In [22]:
weather.precipitation_inches.unique()

array(['0', '0.23', 'T', '0.01', '0.28', '0.63', '0.29', '0.06', '0.85',
       '0.09', '0.64', '0.42', '0.35', '0.43', '0.22', '0.74', '0.03',
       '0.12', '0.16', '0.49', '0.17', '0.08', '0.04', '0.53', '0.07',
       '0.02', '0.83', '1.06', '1.71', '0.37', '0.27', '0.45', '0.78',
       '0.88', '0.66', '0.47', '0.1', '0.61', '0.14', '0.05', '0.68',
       '0.97', '0.26', '0.15', '0.87', '0.57', '0.69', '0.32', '0.21',
       '0.24', '0.52', '0.36', '0.33', '0.25', '0.11', '0.2', '1.18',
       '1.43', '3.12', '0.48', '0.19', '1.09', '0.65', '0.13', '0.91',
       '0.99', '0.18', '0.4', '1.07', nan, '0.41', '0.34', '1.25', '1.85',
       '3.36', '0.71', '1.3', '0.72', '0.6', '0.51', '1.2', '1.28',
       '3.23', '0.55', '1.26', '0.39'], dtype=object)

In [23]:
#T in precipitation means trace which is almost neglegible <0.01 so imputing with 0.01
weather.precipitation_inches.replace('T', '0.01', inplace=True)

In [24]:
#converting string to float for precipitation.
weather.precipitation_inches = weather.precipitation_inches.astype(float)

In [25]:
#imputing all NaN values by median()
weather['mean_temperature_f'] = weather['mean_temperature_f'].fillna(weather.mean_temperature_f.median())
weather['precipitation_inches'] = weather['precipitation_inches'].fillna(weather.precipitation_inches.median())
weather['mean_visibility_miles'] = weather['mean_visibility_miles'].fillna(weather.mean_visibility_miles.median())
weather['mean_humidity'] = weather['mean_humidity'].fillna(weather.mean_humidity.median())
weather['mean_wind_speed_mph'] = weather['mean_wind_speed_mph'].fillna(weather.mean_wind_speed_mph.median())

In [26]:
weather.isnull().sum()

date                     0
mean_temperature_f       0
mean_humidity            0
mean_visibility_miles    0
mean_wind_speed_mph      0
precipitation_inches     0
events                   0
zip_code                 0
dtype: int64

In [27]:
weather.sort_values(by='date').head()

,date,mean_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,events,zip_code
0,2013-08-29,68.0,75.0,10.0,11.0,0.0,Normal,94107
552,2013-08-29,70.0,72.0,10.0,5.0,0.0,Normal,94041
368,2013-08-29,71.0,71.0,10.0,8.0,0.0,Normal,94301
184,2013-08-29,71.0,79.0,10.0,6.0,0.0,Normal,94063
736,2013-08-29,72.0,69.0,10.0,7.0,0.0,Normal,95113


### Merging 'weather' with 'trip_cnt' and creating a new dataframe 'train'

In [33]:
#Aggregating the weather data by date and zip_code
weather2 = weather.groupby(['date'], as_index= False).agg({'mean_temperature_f':'mean','mean_humidity':'mean','mean_visibility_miles':'mean','mean_wind_speed_mph':'mean','precipitation_inches':'mean','events':'max'})
#trip_cnt.rename(columns = {'trip_id':'trips'}, inplace=True)
weather2.events.value_counts()

Normal               601
Rain                 124
Fog                    4
Rain-Thunderstorm      3
Fog-Rain               1
Name: events, dtype: int64

In [35]:
#merge the total trips and weather data frames on date.
train = weather2.merge(trip_cnt, left_on=[weather2.date], right_on=[trip_cnt.date])
train.head()

,key_0,date_x,mean_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,events,date_y,trips
0,2013-08-29,2013-08-29,70.4,73.2,10.0,7.4,0.000,Normal,2013-08-29,748
1,2013-08-30,2013-08-30,73.0,68.8,10.0,5.6,0.000,Normal,2013-08-30,714
2,2013-08-31,2013-08-31,68.0,70.8,10.0,7.4,0.000,Normal,2013-08-31,640
3,2013-09-01,2013-09-01,70.0,66.8,10.0,7.2,0.000,Normal,2013-09-01,706
4,2013-09-02,2013-09-02,70.8,74.0,10.0,7.4,0.004,Rain,2013-09-02,661


In [36]:
#dropping duplicate keys
train.drop(['key_0','date_y'], axis=1, inplace=True)
train.rename(columns={'date_x':'date'}, inplace=True)
train.head()

,date,mean_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,events,trips
0,2013-08-29,70.4,73.2,10.0,7.4,0.000,Normal,748
1,2013-08-30,73.0,68.8,10.0,5.6,0.000,Normal,714
2,2013-08-31,68.0,70.8,10.0,7.4,0.000,Normal,640
3,2013-09-01,70.0,66.8,10.0,7.2,0.000,Normal,706
4,2013-09-02,70.8,74.0,10.0,7.4,0.004,Rain,661


In [37]:
#Convert date to year, month, weekday (0 = Monday, 1 = Tuesday...)
train['year'] = pd.to_datetime(train['date']).dt.year
train['month'] = pd.to_datetime(train['date']).dt.month
train['weekday'] = pd.to_datetime(train['date']).dt.weekday

In [41]:
train.head()

,date,mean_temperature_f,mean_humidity,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,events,trips,year,month,weekday
0,2013-08-29,70.4,73.2,10.0,7.4,0.000,Normal,748,2013,8,3
1,2013-08-30,73.0,68.8,10.0,5.6,0.000,Normal,714,2013,8,4
2,2013-08-31,68.0,70.8,10.0,7.4,0.000,Normal,640,2013,8,5
3,2013-09-01,70.0,66.8,10.0,7.2,0.000,Normal,706,2013,9,6
4,2013-09-02,70.8,74.0,10.0,7.4,0.004,Rain,661,2013,9,0


In [195]:
train.to_csv('train.csv')

## Modelling

#### K-Means Clustering: We created clusters for different weather conditions and looked at the number of trips for each of these clusters. To come up with the optimal  number of clusters here, we used cluster inertia as a benchmark which is the sum of squared distances of sample to closest centroid of a cluster.

In [31]:
from scipy.spatial.distance import cdist, pdist
from sklearn.cluster import KMeans

In [14]:
data = pd.read_csv(r"C:\Users\ujjwa\OneDrive\Desktop\Github codes\Academic Projects\Data Mining\sf-bay-area-bike-share\train.csv")
print(data.columns)

Index(['Unnamed: 0', 'date', 'mean_temperature_f', 'mean_humidity',
       'mean_visibility_miles', 'mean_wind_speed_mph', 'precipitation_inches',
       'events', 'Trips', 'total_docks', 'year', 'month', 'weekday'],
      dtype='object')


In [17]:
l = data.drop(columns = ['Unnamed: 0', 'date', 'events', 'Trips', 'total_docks', 'year', 'month', 'weekday'])

In [24]:
X = np.array(l)
Y = np.array(data['Trips'])

In [35]:
for i in range(1,16):
    kmeans = KMeans(n_clusters = i).fit(X)
    #kmeans = kmeans.
    print(kmeans.inertia_)

110471.53059399727
68744.90287410936
50402.793109118415
38794.64976983884
33170.78053860908
28925.667780135558
25952.502835931882
23605.986407973003
21793.388857707345
20219.13795382104
18906.792786587903
17987.154270756742
16835.20523687941
15929.376369814643
14904.84337303433


In [52]:
kmeans = KMeans(n_clusters = 6).fit(X)
C =  kmeans.cluster_centers_
print(C)

[[5.33369565e+01 6.01956522e+01 9.73913043e+00 5.38043478e+00
  5.43478261e-03]
 [6.54358974e+01 7.20307692e+01 9.81538462e+00 1.00974359e+01
  1.07692308e-02]
 [5.64951456e+01 8.34368932e+01 8.15533981e+00 7.06796117e+00
  1.55436893e-01]
 [5.65819209e+01 7.15084746e+01 9.61581921e+00 8.05084746e+00
  3.26553672e-02]
 [6.32916667e+01 4.57291667e+01 9.97916667e+00 6.52083333e+00
  2.08333333e-04]
 [6.57118644e+01 6.27627119e+01 9.87288136e+00 9.22033898e+00
  7.37288136e-03]]


In [53]:
data['Assignments'] = kmeans.labels_
new_DF = data[['Assignments', 'Trips']]
output = new_DF.groupby(['Assignments'], as_index = False).agg({'Trips':'mean'})
mydf = pd.DataFrame(C, columns = l.columns)

In [54]:
output

,Assignments,Trips
0,0,818.152174
1,1,915.435897
2,2,962.669903
3,3,869.073446
4,4,1014.333333
5,5,970.423729


In [ ]:
mydf.to_csv("Regression_input.csv")

We can see that after the 6th cluster the inertia gets stable so we consider 6 as the value.

For these 6 clusters we have 6 centroids which represent the weather condition in each of these clusters. 

Output: Aggregating the trips as mean for each cluster, we found that the cluster no 3 has the least number of trips which suggests that the weather conditions for this cluster might be poor which, but before we conclude anything, our experiment requires further validation. 

We used Azure ML studio to validate our hypothesis using linear regression, Decision forest regression, and Neural network algorithms. As, Decision forest gave best accuracy, the centroid of cluster served as in imput to Decision forest which predicted the number of trips for that weather condition. 

Results – 
 
1. Looking at these outputs we see that the number of trips in the cluster 3 are the lowest for both models where RMSE is 398 and 440 for decision forest and linear regression respectively. 
 
2. We also tried neural network regression but due RMSE being on the higher side, we ended up rejecting this algorithm. 

## Results and Recommendations

#### Decision Forest and Linear Regression predicted the least number of trips in same cluster as seen in K-Means, which validates our theory that certain weather conditions, most often the extreme ones can cause the trips to drop significantly, and this can take a hit on the business. 
 
#### Companies can use the weather forecast analysis to be prepared of the approaching bad/poor weather conditions and should invest in marketing strategies such as: 
 
    ➢ Rolling out promotional offers for eg. discounts or free rentals for a certain period would be beneficial for their sustainability. 

    ➢ Launch hiking/trekking initiatives with minimal registration costs to attract trek lovers. 

    ➢ Incorporate some safety assurance guideline or an accidental coverage plans and be equipped with the latest technology to track bikes. This will assure its customers and give them the confidence to ride more. 